In [ ]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
# connection au WS
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline
from azureml.data import OutputFileDatasetConfig

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

ws = Workspace.from_config() 
experiment = Experiment(ws, 'Firstpipeline') 


dataprep_step = PythonScriptStep(
    name="prep_data",
    script_name="scripts/datacleaning_scipt1.py",
    source_directory=".",
    compute_target='jcharley1',
    #arguments=["--prepped_data_path", prepped_data_path],
    #inputs=[input_dataset.as_named_input('raw_data').as_mount() ]
    )

OutputFileDatasetConfig(name="output_path")

train_step = PythonScriptStep(
    name="train",
    script_name="scripts/model_script2.py",
    compute_target='jcharley1',
    #arguments=["--prepped_data", prepped_data],
    source_directory="."
)
steps = [dataprep_step, train_step ]


pipeline = Pipeline(workspace=ws, steps=steps)

pipeline_run = experiment.submit(pipeline)
pipeline_run.wait_for_completion()